In [ ]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from urllib.parse import unquote
import matplotlib.pyplot as plt 
import re 
from highlight_text import fig_text
from mplsoccer import PyPizza, FontManager

def extract_player_name(player_url):
    return unquote(player_url.split('/')[-1]).replace('-Scouting-Report', '').replace('-', ' ')

params = []
params_p2 = []
values = []
values_p2 = []
params_offset = []

def scrape_player_data(player_url, params, values):
    player_name = extract_player_name(player_url)

    # Set up a headless browser using Selenium
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)

    try:
        # Open the player URL in the browser
        driver.get(player_url)
        time.sleep(15)  # Allow time for JavaScript to load (adjust as needed)

        # Get the page source after JavaScript execution
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        target_data_rows = [102, 117, 119, 113, 141, 139, 142, 127, 132, 83, 60, 56, 58, 70]
        for row_number in target_data_rows:
            tr_element = soup.find('tr', {'data-row': str(row_number)})

            if tr_element:
                stat_name = tr_element.find('th', {'data-stat': 'statistic'}).text.strip()
                td_element = tr_element.find('td', {'data-stat': 'percentile'})

                if td_element:
                    percentile_value = td_element.find('div').text.strip()
                    
                    if player_url == player_urls[0]: 
                        params.append(stat_name)
                    elif player_url == player_urls[1]:
                        params_p2.append(stat_name)
                        
                    values.append(int(percentile_value))
                    
                    
                    print(f"{player_name} {stat_name} - Percentile: {percentile_value}%")
                else:
                    print(f"Error: Could not find 'percentile' data in row {row_number}")
            else:
                print(f"Error: Could not find row {row_number}")

    finally:
        # Close the browser
        driver.quit()

def line_breaks(params):
    return [re.sub(r'(?<= )([A-Z])', r'\n\1', stat_name) for stat_name in params]


player_urls = ['https://fbref.com/en/players/d9565625/scout/12192/Diogo-Dalot-Scouting-Report', 
            'https://fbref.com/en/players/9e525177/scout/12192/Aaron-Wan-Bissaka-Scouting-Report']

player_names = []

# Iterate through player URLs
for player_url in player_urls:
    player_name = extract_player_name(player_url)
    player_names.append(player_name)  # Append each player name to the list

    if player_url == player_urls[0]:
        scrape_player_data(player_url, params, values)
    elif player_url == player_urls[1]:
        scrape_player_data(player_url, params, values_p2)



formatted_params = line_breaks(params)
formatted_params_p2 = line_breaks(params_p2)

In [ ]:
#print(len(params))
#print(len(formatted_params))
#print(len(values))
#print(len(values_p2))
#print(player_names[0])
#print(player_names[1]) 


In [ ]:
font_normal = FontManager('https://github.com/google/fonts/raw/main/ofl/montserrat/Montserrat%5Bwght%5D.ttf')
font_italic = FontManager('https://github.com/google/fonts/raw/main/ofl/montserrat/Montserrat-Italic%5Bwght%5D.ttf')
font_bold = FontManager('https://github.com/google/fonts/raw/main/ofl/montserrat/Montserrat%5Bwght%5D.ttf') 

for v1, v2 in zip(values, values_p2):
    if abs(v1 -v2) <= 7:
        params_offset.append(True)
    else: 
        params_offset.append(False)
        



# instantiate PyPizza class
baker = PyPizza(
    params = formatted_params,
    background_color="#011627",     # background color
    straight_line_color="#FFFFFF",  # color for straight lines
    straight_line_lw=2,             # linewidth for straight lines
    last_circle_lw=2,               # linewidth of last circle
    last_circle_color="#FFFFFF",    # color of last circle
    other_circle_ls="-",            # linestyle for other circles
    other_circle_lw=1,               # linewidth for other circles
    other_circle_color="#FFFFFF",
)

# plot pizza
fig, ax = baker.make_pizza(
    values,                     # list of values
    compare_values=values_p2,    # comparison values
    figsize=(34,34),             # adjust figsize according to your need
    kwargs_slices=dict(
        facecolor="#8ceceb", edgecolor="#FFFFFF",
        zorder=2, linewidth=1,
    ),                          # values to be used when plotting slices
    kwargs_compare=dict(
        facecolor="#de1b5e", edgecolor="#FFFFFF",
        zorder=2, linewidth=1,
    ),
    kwargs_params=dict(
        color="#FFFFFF", fontsize=44,
        fontproperties=font_bold.prop, va="center"
    ),                          # values to be used when adding parameter
    kwargs_values=dict(
        color="#011627", fontsize=44,
        fontproperties=font_bold.prop, zorder=3,
        bbox=dict(
            edgecolor="#011627",
            facecolor="#8ceceb",
            boxstyle="round,pad=0.2", lw=1,
        )
    ),                          # values to be used when adding parameter-values labels
        kwargs_compare_values=dict(
        color="#FFFFFF", fontsize=40, fontproperties=font_normal.prop, zorder=3,
        bbox=dict(edgecolor="#011627", facecolor="#9b1342", boxstyle="round,pad=0.2", lw=1)
    ),                          # values to be used when adding parameter-values labels
)

# adjust text for comparison-values-text
baker.adjust_texts(params_offset, offset=-0.15, adj_comp_values=True)

# add title
fig_text(
    0.515, 0.99, f"<{player_names[0]}> vs <{player_names[1]}>", size=70, fig=fig,
    highlight_textprops=[{"color": '#8ceceb'}, {"color": '#de1b5e'}],
    ha="center", fontproperties=font_bold.prop, color="#FFFFFF"
)

# add subtitle
fig.text(
    0.515, 0.942,
    "Percentile Rank vs Top-Five League Fullbacks | Last 365 Days",
    size=35,
    ha="center", fontproperties=font_bold.prop, color="#FFFFFF"
)


plt.show()
